In [59]:
import pandas as pd
import os
import json
import sys
import time
import numpy as np
from collections import defaultdict
import os 


os.chdir(r'F:\360MoveData\Users\Administrator\Desktop')
df = pd.read_excel(r'data.xlsx',sheet_name=1)
df.head()


result = []
for i, row in df.iterrows():
    colums = row.index.values
    d= defaultdict(list)
    for colum in colums:
        try:
            d[colum] = json.loads(row[colum])
            data=json.loads(row[colum])['data']
            info=json.loads(row[colum])['info']
            label=json.loads(row[colum])['type']
            colum_dest = colum
            if label=='TXT':
                if isinstance(data, list):
                    for i in data:
                        if isinstance(i, dict):
                            d['start'].append(i.get('start'))
                            d['columnIndex'].append(i.get('columnIndex'))
                            d['end'].append(i.get('end'))
                            d['annotateType'].append(i.get('annotateType'))
                            if isinstance(i.get('info'), list):
                                for j in i.get('info'):
                                    if isinstance(j.get('val'), list):
                                        for k in j.get('val'):
                                            if isinstance(k, dict):
                                                d['data-realLabelValue'].append(k.get('realLabelValue'))
                if isinstance(info, list):
                     for g in info:
                        for k in g.get('val'):
                            if isinstance(k, dict):
                                d['info-realLabelValue'].append(k.get('realLabelValue'))
                            else:
                                d['info-val'].append(k)
                                
            elif label=='PHOTO':
                if isinstance(data, list):
                    for i in data:
                        if isinstance(i, dict):
                            d['shapeType'].append(i.get('shapeType'))
                            d['points'].append(i.get('points'))
                            if isinstance(i.get('info'), list):
                                    for j in i.get('info'):
                                        if isinstance(j.get('val'), list):
                                            for k in j.get('val'):
                                                if isinstance(k, dict):
                                                    d['data-realLabelValue'].append(k.get('realLabelValue'))
                if isinstance(info, list):
                     for g in info:
                        if isinstance(g.get('val'),list):
                            for k in g.get('val'):
                                if isinstance(k,dict):
                                    d['info-realLabelValue'].append(k.get('realLabelValue'))
                                else:
                                    d['info-realLabelValue'].append(k)
                        else:
                            if g:
                                d['info-val'].append(g.get('val'))
                            
        except:
            d[colum] = row[colum]
    result.append(d)

df = pd.DataFrame(result)   

if label=='TXT':
    result1=[]
    for j, row in df.iterrows():

        if isinstance(row.start, list):
            for i,(start,end) in enumerate(zip(row.start,row.end)):
                row[f'结果{i+1}']=row['标注人员'][start:end]

            result1.append(row)
        else:
            result1.append(row)


    df=pd.DataFrame(result1)


    
    
    df1=pd.DataFrame()
    for i in ['info-realLabelValue','start','columnIndex','end','annotateType','data-realLabelValue']:
        m=df[i].apply(lambda x : ",".join('%s' %i for i in x) if  isinstance(x, list)  else x)
        m=m.str.split(',',expand=True)
        m.columns = [i + str(col)  if  str(col).isdigit() else col  for col in m.columns]
        df1=pd.concat([df1,m],axis=1)

    df=df.join(df1)
    df.drop(columns=[colum_dest,'info-realLabelValue','start','columnIndex','end','annotateType','data-realLabelValue'], inplace=True)

    
if label=='PHOTO':
    df=df.explode('points')
    df1=pd.DataFrame()
    for i in ['info-realLabelValue','shapeType','data-realLabelValue']:
        m=df[i].apply(lambda x : ",".join('%s' %i for i in x) if  isinstance(x, list)  else x)
        m=m.str.split(',',expand=True)
        m.columns = [i + str(col)  if  str(col).isdigit() else col  for col in m.columns]
        df1=pd.concat([df1,m],axis=1)

    df=df.join(df1)
    df.drop(columns=[colum_dest,'info-realLabelValue','shapeType','data-realLabelValue'], inplace=True)

    
df.replace({None:np.nan},inplace=True)
df.reset_index(drop=True, inplace=True)
df=df.iloc[df.astype(str).drop_duplicates().index]
df.reset_index(drop=True, inplace=True)





df

,标注人员,info-val,结果1,结果2,结果3,info-realLabelValue0,start0,start1,start2,columnIndex0,...,columnIndex2,end0,end1,end2,annotateType0,annotateType1,annotateType2,data-realLabelValue0,data-realLabelValue1,data-realLabelValue2
0,王二打法啊发的大事,[App],NaN,NaN,NaN,生活服务>运营商服务,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,杨六框架开发的沙河口,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,张三的算法框架,NaN,NaN,NaN,NaN,教育培训>翻译,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,李一范德萨发德萨发发喀什酱豆腐,NaN,一范,萨发,萨发,NaN,1,4,7,4,...,4,3,6,9,slot,slot,slot,市区,朝阳区,测试测试
